In [1]:
%matplotlib notebook
import os
from adm_library import *

## 0 Parameter definitions

### 0.1 Ingesting the [X,Y,Z,Rx,Ry] values from Tyler's spreadsheet

In [2]:
fname = 'files/Cal5DoF05222023-post FDPR V3.xlsx'
sheetname = 'Input for Tyler analysis'
#fname = 'files/5DoF inputs - sandbox.xlsx'
#sheetname = 'Input for Tyler analysis'

spreadsheet = pd.read_excel(fname,
                            sheet_name=sheetname,skiprows=1,usecols='B:N')

focal_length = spreadsheet.iloc[0][0]-110 # focal length is track length -110mm, per TG
sMPA_to_WCS = spreadsheet.iloc[3,:3].astype(float)
sMPA_angle_to_WCS_deg = spreadsheet.iloc[6,:3].astype(float)
GSA_angle_WCS_deg = spreadsheet.iloc[7][0]
sMask=np.array(spreadsheet.iloc[10,1:4]).astype(float)

spreadsheet = pd.read_excel(fname,
                            sheet_name=sheetname,skiprows=15,usecols='B:N')
pose_actual = spreadsheet.iloc[0:5,0:6]
pose_actual = pose_actual.set_index('Name').sort_index()
pose_actual.columns = ['X','Y','Z','Rx','Ry']
X_5DOF,Y_5DOF,Z_5DOF,rx_deg_5DOF,ry_deg_5DOF = [[*pose_actual['X'].values.astype(float)],[*pose_actual['Y'].values.astype(float)],[*pose_actual['Z'].values.astype(float)],
                                                [*pose_actual['Rx'].values.astype(float)],[*pose_actual['Ry'].values.astype(float)]]
pose_encoders = spreadsheet.iloc[0:5,7:13]
pose_encoders = pose_encoders.set_index('Name.1').sort_index()

print('Provided 5DOF position in real space: \n',pose_actual,'\n')
print('Provided 5DOF encoder values: \n',pose_encoders)


Provided 5DOF position in real space: 
                X          Y          Z        Rx         Ry
Name                                                       
PR1     1.957073   0.213787   0.797286  0.026155   0.003663
PR2  -108.548109 -46.839383  23.430574 -4.906209  10.653646
PR3   112.267199 -46.530542  23.278306 -4.873173 -10.626984
PR4   104.116729  58.616016  31.900208  5.293205 -10.348396
PR5  -100.390912  58.423929  31.865299  5.273991  10.373671 

Provided 5DOF encoder values: 
                X.1        Y.1        Z.1      Rx.1       Ry.1
Name.1                                                       
PR1       1.933524   0.250412   0.783184  0.026885   0.005745
PR2    -108.452857 -46.898136  23.320940 -4.906167  10.652669
PR3     112.219072 -46.231812  23.378512 -4.871664 -10.626735
PR4     103.923673  58.840103  32.096062  5.298097 -10.345960
PR5    -100.448696  58.292823  31.861952  5.273415  10.374527


### 0.1.1 If both the pose_actual and pose_encoder arrays are provided in the spreadsheet above, convert between the two to make sure they both agree. 

In [3]:
#No point doing the following if one or the other array is completely empty
if (not any(np.isfinite(pose_encoders.values.astype(float)).ravel())) and (not any(np.isfinite(pose_actual.values.astype(float)).ravel())):

    print('(Calculated 5DOF position) - (Actual 5DOF position (from the spreadsheet)): ')
    calculated_pose_actual = pd.DataFrame(columns=pose_actual.columns)
    for index in [val for val in pose_actual.index if ('PR' in val) or ('PD' in val)]:
        calculated_pose_actual = pd.concat((calculated_pose_actual,calculate_5DOF_from_encoders(pd.DataFrame(pose_encoders.loc[index]).T)))
    print(calculated_pose_actual-pose_actual,'\n\n')
    
    print('(Calculated 5DOF encoders) - (Actual 5DOF encoders (from the spreadsheet)): ')
    calculated_pose_encoders = pd.DataFrame(columns=pose_encoders.columns)
    for index in [val for val in pose_encoders.index if ('PR' in val) or ('PD' in val)]:
        calculated_pose_encoders = pd.concat((calculated_pose_encoders,calculate_encoders_from_5DOF(pd.DataFrame(pose_actual.loc[index]).T)))
    print(calculated_pose_encoders-pose_encoders)

### 0.2 If the "pose_actual" section of the supplied spreadsheet was empty, build a new "pose_actual" array based on the input "pose_encoder" array.

In [4]:
if not any(np.isfinite(pose_actual.values.astype(float)).ravel()):
    pose_actual = convert_pose_encoders_to_pose_actual(pose_encoders)
    print('NEWLY calculated 5DOF position in real space based on recorded 5DOF encoder values: \n',pose_actual)

# 1. Updating Poses with ADM Data

## 1.1 ADM Baseline  

#### 1.1.1 Input your baseline ADM values

In [5]:
#p_null_PAT_baseline_encoder = pd.DataFrame(dict({'X':88.435, 'Y':109.266,'Z':0,
#                                                  'Rx':0.1018,'Ry':0.0856,
#                                                  'z_sMATF':4.027174*1000.,
#                                                  'z_PATB':3.455684*1000.,
#                                                  'z_type':'retro',
#                                                  'sMATF AC AZ':0.0041,
#                                                  'sMATF AC EL':0.006,
#                                                  'PATB AC AZ':0.0041,
#                                                  'PATB AC EL':0.0035,
#                                                  'sMATF Pri LED X':811.9,
#                                                  'sMATF Pri LED Y':604.6,
#                                                  'PATB Pri LED X':812.6,
#                                                  'PATB Pri LED Y':605.9,
#                                                  'date':'4/1/2023'
#                                                 }),index=['PATB'])
pat_target = 'B'
z_target = 'mirror'
optimize_PAT_for_imperfect_nulling = True

p_null_PAT_baseline_encoder = get_data_from_ADM_log('Ambient (2nd pass)',z_type=z_target,pat_target = pat_target,index_name='PAT'+pat_target,print_details=True) 
p_null_PAT_baseline = calculate_5DOF_from_encoders(p_null_PAT_baseline_encoder)

if pat_target=='B':
    p_null_offset = np.array([-90.,-110.,-10.]) #PATB is not on the POA boresight. It is offset by this much
elif pat_target=='A':
    p_null_offset = np.array([90.,-110.,-10.])
p_null_PAT_baseline_encoder


X                               88.711
Y                               108.18
Z                                  0.0
Rx                              0.1056
Ry                              0.0868
z_sMATF                        4021.75
z_PAT                          3472.25
z_type                          mirror
sMATF AC AZ                    -0.0098
sMATF AC EL                     0.0108
PAT AC AZ                      -0.0098
PAT AC EL                       0.0091
sMATF Pri LED X                813.891
sMATF Pri LED Y                594.714
PAT Pri LED X                  812.836
PAT Pri LED Y                  594.935
date               2023-05-24 00:00:00
Name: PATB, dtype: object


,X,Y,Z,Rx,Ry,z_sMATF,z_PAT,z_type,sMATF AC AZ,sMATF AC EL,PAT AC AZ,PAT AC EL,sMATF Pri LED X,sMATF Pri LED Y,PAT Pri LED X,PAT Pri LED Y,date
PATB,88.711,108.18,0.0,0.1056,0.0868,4021.75,3472.25,mirror,-0.0098,0.0108,-0.0098,0.0091,813.891,594.714,812.836,594.935,2023-05-24


### 1.1.2 Update the PAT pose to account for imperfect nulling.

In [6]:
#Apply a correction to account for imperfect nulling between PATB/sMATF and overwrite the values in the p_null_PAT_baseline_encoder DataFrame with the optimal ones.

#Save a copy of the p_null_PATB dataframe, which we'll write to an Excel file later
p_null_PAT_baseline_encoder_original = p_null_PAT_baseline_encoder.copy()
if optimize_PAT_for_imperfect_nulling == True:
    print(p_null_PAT_baseline_encoder.squeeze(),'\n')
    p_null_PAT_baseline_encoder = optimize_p_null_PAT_encoders(p_null_PAT_baseline_encoder,p_null_offset)
    p_null_PAT_baseline = calculate_5DOF_from_encoders(p_null_PAT_baseline_encoder)
    print('\n',p_null_PAT_baseline_encoder.squeeze())

X                               88.711
Y                               108.18
Z                                  0.0
Rx                              0.1056
Ry                              0.0868
z_sMATF                        4021.75
z_PAT                          3472.25
z_type                          mirror
sMATF AC AZ                    -0.0098
sMATF AC EL                     0.0108
PAT AC AZ                      -0.0098
PAT AC EL                       0.0091
sMATF Pri LED X                813.891
sMATF Pri LED Y                594.714
PAT Pri LED X                  812.836
PAT Pri LED Y                  594.935
date               2023-05-24 00:00:00
Name: PATB, dtype: object 

Starting GSARX/RY:  0.1056 0.0868
Optimal GSARX/RY:  0.1065628919 0.0882010142
Starting HTSA/VTSA:  88.711 108.18
Optimal HTSA/VTSA:  88.75817527999999 108.15797167000001

 X                            88.758175
Y                           108.157972
Z                                  0.0
Rx                 

#### 1.1.5 Form a dataframe with the values supplied so far

In [7]:
#Generate a dataframe where we will store some parameters for each pose
columns = ['X','Y','Z','Rx','Ry']
df = pose_actual
df['color']='crimson'

gsa_rot = R.from_euler('x',GSA_angle_WCS_deg, degrees=True)
df.loc['sMask',['X','Y','Z']] = gsa_rot.apply(sMask)
df.loc['sMask','color'] = 'g'

dz_baseline = p_null_PAT_baseline_encoder.loc['PAT'+pat_target,'z_sMATF'] - \
    p_null_PAT_baseline_encoder.loc['PAT'+pat_target,'z_PAT']
df = pd.concat([df,p_null_PAT_baseline])
print(p_null_PAT_baseline)
df.loc['PAT'+pat_target,'color']='darkblue'

df.loc['sMPA',['X','Y','Z']] = sMPA_to_WCS.values.astype(float)
df.loc['sMPA','color']='purple'

for pose in df.index:
    update_uvec(df,pose,length=focal_length,rotangle=GSA_angle_WCS_deg)
    if 'PDI' in pose:
        df.loc[pose,'color'] = 'yellow'

# create normal vector to sMPA
vec = np.array([0,0,1])
rotmat = R.from_euler('XYZ',sMPA_angle_to_WCS_deg, degrees=True)
df.loc['sMPA',['uvec_X','uvec_Y','uvec_Z']] = rotmat.apply(vec)

#ax = plot_poses(df)
#plot_sMPA(df, ax)
df

              X           Y         Z        Rx        Ry
PATB  89.082777  108.396775 -0.679302  0.105857  0.086117


,X,Y,Z,Rx,Ry,color,uvec_X,uvec_Y,uvec_Z
PR1,1.957073,0.213787,0.797286,0.026155,0.003663,crimson,0.082611,248.260690,1268.049080
PR2,-108.548109,-46.839383,23.430574,-4.906209,10.653646,crimson,238.876843,350.225013,1220.599091
PR3,112.267199,-46.530542,23.278306,-4.873173,-10.626984,crimson,-238.285909,349.551738,1220.907554
PR4,104.116729,58.616016,31.900208,5.293205,-10.348396,crimson,-232.108195,128.680371,1264.574630
PR5,-100.390912,58.423929,31.865299,5.273991,10.373671,crimson,232.668903,129.094033,1264.429420
sMask,1.995,115.232021,588.279458,NaN,NaN,g,NaN,NaN,NaN
PATB,89.082777,108.396775,-0.679302,0.105857,0.086117,darkblue,1.942101,246.496234,1268.391770
sMPA,2.04,248.465,1268.826,NaN,NaN,purple,0.000129,0.411468,0.911424


#### 1.1.6 Calculate the center of the sMATF flat in 5DOF coordinate system (currently assumes PATB is on-axis and aligned with GSA)

In [8]:
start_coords = np.array(df.loc['PAT'+pat_target,['X','Y','Z']])

rx = df.loc['PAT'+pat_target,'Rx']
ry = df.loc['PAT'+pat_target,'Ry']
print(rx,ry)
gsa_rot = R.from_euler('XY',[rx+GSA_angle_WCS_deg,ry], degrees=True)
uvec = np.array([0.,0.,dz_baseline]+p_null_offset)
rot_uvec = gsa_rot.apply(uvec) #np.dot(gsa_rot.as_matrix(),uvec)

sMATF_coords = start_coords+rot_uvec

df.loc['sMATF',['X','Y','Z']]=sMATF_coords
df.loc['sMATF',['Rx','Ry']]=df.loc['PAT'+pat_target,['Rx','Ry']]
df.loc['sMATF','color']='orange'
update_uvec(df,'sMATF',length=dz_baseline,rotangle=GSA_angle_WCS_deg)
#ax = plot_poses(df)
#plot_sMPA(df, ax)
df

0.10585702792202817 0.08611736105627892


,X,Y,Z,Rx,Ry,color,uvec_X,uvec_Y,uvec_Z
PR1,1.957073,0.213787,0.797286,0.026155,0.003663,crimson,0.082611,248.260690,1268.049080
PR2,-108.548109,-46.839383,23.430574,-4.906209,10.653646,crimson,238.876843,350.225013,1220.599091
PR3,112.267199,-46.530542,23.278306,-4.873173,-10.626984,crimson,-238.285909,349.551738,1220.907554
PR4,104.116729,58.616016,31.900208,5.293205,-10.348396,crimson,-232.108195,128.680371,1264.574630
PR5,-100.390912,58.423929,31.865299,5.273991,10.373671,crimson,232.668903,129.094033,1264.429420
sMask,1.995,115.232021,588.279458,NaN,NaN,g,NaN,NaN,NaN
PATB,89.082777,108.396775,-0.679302,0.105857,0.086117,darkblue,1.942101,246.496234,1268.391770
sMPA,2.04,248.465,1268.826,NaN,NaN,purple,0.000129,0.411468,0.911424
sMATF,-0.106236,103.362345,550.029882,0.105857,0.086117,orange,0.825916,104.827302,539.408186


## 1.2 ADM Update

Now the system has undergone some kind of change that will cause the ADM measurements to be different, such as shimming the sWFI or cooling from ambient to operational temps. We need to use the new set of ADM sMATF/PATB measurements to update the calculated poses for PRs 1-5.

Now update all poses from this new PATB pose; this is the recipe that would be followed for real data

#### 1.2.1 Input new ADM data for the sMATF and PATB and convert to 5DOF actual space

In [9]:
#p_null_PAT_update_encoder = pd.DataFrame(dict({'X':88.526, 'Y':108.364,'Z':0,
#                                                  'Rx':0.1287,'Ry':0.0788,
#                                                  'z_sMATF':4.031636*1000.,
#                                                  'z_PATB':3.458517*1000.,
#                                                  'z_type':'retro',
#                                                  'sMATF AC AZ':-0.0022,
#                                                  'sMATF AC EL':0.0042,
#                                                  'PATB AC AZ':-0.0008,
#                                                  'PATB AC EL':0.0001,
#                                                  'sMATF Pri LED X':811.7,
#                                                  'sMATF Pri LED Y':597.6,
#                                                  'PATB Pri LED X':812.1,
#                                                  'PATB Pri LED Y':598.8,
#                                                  'date':'4/3/2023'
#                                                 }),index=['PATB_update'])
p_null_PAT_update_encoder = get_data_from_ADM_log('Ambient (post-shim2 v2)',z_type=z_target,pat_target = pat_target,index_name='PAT'+pat_target+'_update',print_details=True) 
p_null_PAT_update = calculate_5DOF_from_encoders(p_null_PAT_update_encoder)
p_null_PAT_update

X                               88.936
Y                               107.73
Z                                  0.0
Rx                              0.0143
Ry                              0.0735
z_sMATF                       4021.463
z_PAT                         3472.261
z_type                          mirror
sMATF AC AZ                    -0.0122
sMATF AC EL                     0.0056
PAT AC AZ                      -0.0105
PAT AC EL                       0.0049
sMATF Pri LED X                  813.7
sMATF Pri LED Y                  595.6
PAT Pri LED X                   813.78
PAT Pri LED Y                    596.6
date               2023-05-26 00:00:00
Name: PATB_update, dtype: object


,X,Y,Z,Rx,Ry
PATB_update,89.260112,107.967819,-0.6781,0.01357,0.071422


### 1.2.2 Update the PATB pose to account for imperfect nulling

In [10]:
#Apply a correction to account for imperfect nulling between PATB/sMATF and overwrite the values in the p_null_PAT_baseline_encoder DataFrame with the optimal ones.

#Save a copy of the p_null_PATB dataframe, which we'll write to an Excel file later
p_null_PAT_update_encoder_original = p_null_PAT_update_encoder.copy()

if optimize_PAT_for_imperfect_nulling == True:
    p_null_PAT_update_encoder = optimize_p_null_PAT_encoders(p_null_PAT_update_encoder,p_null_offset)
    p_null_PAT_update = calculate_5DOF_from_encoders(p_null_PAT_update_encoder)

Starting GSARX/RY:  0.0143 0.0735
Optimal GSARX/RY:  0.0132954707 0.0750397852
Starting HTSA/VTSA:  88.936 107.73
Optimal HTSA/VTSA:  88.9624476 107.7709376


#### 1.2.2 Use the newly-provided data to calculate an updated position for sMATF

In [11]:
dz_update = p_null_PAT_update_encoder.loc['PAT'+pat_target+'_update','z_sMATF'] - \
    p_null_PAT_update_encoder.loc['PAT'+pat_target+'_update','z_PAT']
df = pd.concat([df,p_null_PAT_update])
df.loc['PAT'+pat_target+'_update','color'] = 'purple'
update_uvec(df,'PAT'+pat_target+'_update',length=focal_length,rotangle=GSA_angle_WCS_deg)

start_coords = np.array(df.loc['PAT'+pat_target+'_update',['X','Y','Z']])

rx = df.loc['PAT'+pat_target+'_update','Rx']
ry = df.loc['PAT'+pat_target+'_update','Ry']
gsa_rot = R.from_euler('XY',[rx+GSA_angle_WCS_deg,ry], degrees=True)
uvec = [0.,0.,dz_update]+p_null_offset # in GSA CS
rot_uvec = gsa_rot.apply(uvec) #np.dot(gsa_rot.as_matrix(),uvec)

sMATF_update_coords = start_coords+rot_uvec

df.loc['sMATF_update',['X','Y','Z']]=sMATF_update_coords
df.loc['sMATF_update',['Rx','Ry']]=df.loc['PAT'+pat_target+'_update',['Rx','Ry']]
df.loc['sMATF_update','color']='brown'
update_uvec(df,'sMATF_update',length=dz_update,rotangle=GSA_angle_WCS_deg)
#ax = plot_poses(df)
#plot_sMPA(df, ax)
print('Baseline poses')
df


Baseline poses


,X,Y,Z,Rx,Ry,color,uvec_X,uvec_Y,uvec_Z
PR1,1.957073,0.213787,0.797286,0.026155,0.003663,crimson,0.082611,248.260690,1268.049080
PR2,-108.548109,-46.839383,23.430574,-4.906209,10.653646,crimson,238.876843,350.225013,1220.599091
PR3,112.267199,-46.530542,23.278306,-4.873173,-10.626984,crimson,-238.285909,349.551738,1220.907554
PR4,104.116729,58.616016,31.900208,5.293205,-10.348396,crimson,-232.108195,128.680371,1264.574630
PR5,-100.390912,58.423929,31.865299,5.273991,10.373671,crimson,232.668903,129.094033,1264.429420
sMask,1.995,115.232021,588.279458,NaN,NaN,g,NaN,NaN,NaN
PATB,89.082777,108.396775,-0.679302,0.105857,0.086117,darkblue,1.942101,246.496234,1268.391770
sMPA,2.04,248.465,1268.826,NaN,NaN,purple,0.000129,0.411468,0.911424
sMATF,-0.106236,103.362345,550.029882,0.105857,0.086117,orange,0.825916,104.827302,539.408186
PATB_update,89.286729,108.008791,-0.678405,0.012566,0.072961,purple,1.645411,248.561234,1267.989137


In [12]:
#How did the distance between PATB and sMATF change from pre- to post-environment shift?
print('PAT'+pat_target+'->sMATF distance changed by: ',dz_update-dz_baseline, ' mm')

PATB->sMATF distance changed by:  -0.29400484147527095  mm


## 1.3 Calculate deltas between baseline & updated ADM measurements

In [13]:
sMATF_delta = df.loc['sMATF_update',['X','Y','Z']] - df.loc['sMATF',['X','Y','Z']]
print('sMATF (X,Y,Z) deltas: \n',sMATF_delta,'\n')

patb_delta = df.loc['PAT'+pat_target+'_update',['X','Y','Z','Rx','Ry']] - df.loc['PAT'+pat_target,['X','Y','Z','Rx','Ry']]
#patb_drxry = df.loc['PATB_update',['Rx','Ry']] - df.loc['PATB',['Rx','Ry']]
print('PAT'+pat_target+' (X,Y,Z,Rx,Ry) deltas: \n',patb_delta,'\n')


sMATF (X,Y,Z) deltas: 
 X    0.079673
Y     0.44821
Z    -0.30026
dtype: object 

PATB (X,Y,Z,Rx,Ry) deltas: 
 X     0.203953
Y    -0.387984
Z     0.000896
Rx   -0.093291
Ry   -0.013156
dtype: object 



## 1.4 Transform old poses to the new sMATF reference frame

### 1.4.1 Apply a translation to all poses

In [14]:
df_update = df.copy()
T = df.loc['sMATF_update',['X','Y','Z']]
df_update[['X','Y','Z']] -= T
print('Translation required to put sMATF_update at the origin: \n',T)

Translation required to put sMATF_update at the origin: 
 X     -0.026563
Y    103.810556
Z    549.729621
Name: sMATF_update, dtype: object


### 1.4.2 Now perform 3 rotations to all the poses:
1. To align to the sMATF frame
2. To rotate everything by delta(Rx,Ry)
3. To undo #1

In [15]:
update_rot1 = R.from_euler('XY',[df.loc['PAT'+pat_target,'Rx']+GSA_angle_WCS_deg,df.loc['PAT'+pat_target,'Ry']], degrees=True)
update_rot2 = R.from_euler('XY',[patb_delta.loc['Rx'],patb_delta.loc['Ry']], degrees=True)
update_rot_combined = update_rot1 * update_rot2 * update_rot1.inv()
                                
for pose in df_update.index:
                                
    df_update.loc[pose,['X','Y','Z']]=update_rot_combined.apply(df_update.loc[pose,['X','Y','Z']].astype(float)) #np.dot(update_rot.as_matrix(),df_update.loc[pose,['X','Y','Z']])
    df_update.loc[pose,['uvec_X','uvec_Y','uvec_Z']] = update_rot_combined.apply(df_update.loc[pose,['uvec_X','uvec_Y','uvec_Z']].astype(float)) 
    if pose not in ['sMask','sMPA']:
        update_RxRy(df_update,pose,GSA_angle_WCS_deg)


### 1.4.3 Translate back to the 5DOF frame by undoing the translation from Cell 1.4.1, followed by adding the sMATF delta(X,Y,Z) offset from Cell 1.3

In [16]:
df_update[['X','Y','Z']] += T
df_update.loc[df_update.index[:12],['X','Y','Z']] += update_rot_combined.apply(sMATF_delta.astype(float)) #np.dot(update_rot.as_matrix(),delta)
#ax = plot_poses(df_update)
#plot_sMPA(df_update, ax)
print('Updated poses')
df_update

Updated poses


,X,Y,Z,Rx,Ry,color,uvec_X,uvec_Y,uvec_Z
PR1,2.165053,-0.232049,0.666182,-0.067136,-0.009493,crimson,-0.214086,250.325045,1267.643157
PR2,-108.343044,-47.25345,23.351206,-4.999311,10.640528,crimson,238.586102,352.223094,1220.080904
PR3,112.472278,-46.934574,23.248087,-4.966653,-10.640103,crimson,-238.576676,351.528100,1220.283169
PR4,104.315011,58.225504,31.69694,5.200157,-10.361485,crimson,-232.398578,130.728413,1264.311208
PR5,-100.192608,58.023836,31.61636,5.180457,10.360581,crimson,232.378520,131.163485,1264.269833
sMask,2.065521,115.742593,587.960292,NaN,NaN,g,NaN,NaN,NaN
PATB,89.286087,107.952449,-0.966961,0.012566,0.072961,darkblue,1.645409,248.561236,1267.989137
sMPA,1.951287,250.083486,1268.288989,NaN,NaN,purple,-0.000095,0.412952,0.910753
sMATF,-0.026563,103.810556,549.729621,0.012566,0.072961,orange,0.699742,105.705484,539.236959
PATB_update,89.490057,107.564476,-0.965387,-0.080726,0.059805,purple,1.348714,250.625563,1267.583076


# 1.4 >>>>Results<<<<

#### 1.4.1 Converting poses from real space to encoder space

In [17]:
df_update_encoder_space = convert_df_to_encoder_space(df_update)
print('Poses calculated using PAT'+pat_target+' and the PAT '+z_target)
print('Updated Poses (in encoder space): \n',df_update_encoder_space,'\n\n')

print('Baseline Poses (in encoder space), just for easy reference: \n',pose_encoders)

Poses calculated using PATB and the PAT mirror
Updated Poses (in encoder space): 
               X          Y          Z        Rx         Ry
PR1    2.141982  -0.196776   0.653342 -0.066383  -0.007416
PR2 -108.247274 -47.313599  23.242300 -4.999362  10.639518
PR3  112.424549 -46.638937  23.351032 -4.965268 -10.639897
PR4  104.121579  58.449922  31.893888  5.204977 -10.359096
PR5 -100.249842  57.892074  31.613340  5.179841  10.361402 


Baseline Poses (in encoder space), just for easy reference: 
                X.1        Y.1        Z.1      Rx.1       Ry.1
Name.1                                                       
PR1       1.933524   0.250412   0.783184  0.026885   0.005745
PR2    -108.452857 -46.898136  23.320940 -4.906167  10.652669
PR3     112.219072 -46.231812  23.378512 -4.871664 -10.626735
PR4     103.923673  58.840103  32.096062  5.298097 -10.345960
PR5    -100.448696  58.292823  31.861952  5.273415  10.374527


#### (OPTIONAL) 1.4.2 Display some other tables that may be helpful for diagnostics

In [18]:
print_extra_diagnostics = False
poses_to_display = [val for val in df.index if 'PDI' in val or 'PR' in val]+['sMask', 'sMPA']

if print_extra_diagnostics == True:
    print('Baseline Poses (in real space): \n',df.loc[poses_to_display,['X','Y','Z','Rx','Ry']],'\n\n')
    print('Updated Poses (in real space): \n',df_update.loc[poses_to_display,['X','Y','Z','Rx','Ry']], '\n\n')
    print('Deltas between Baseline/Updated poses (in real space): \n',-df.loc[poses_to_display,['X','Y','Z','Rx','Ry']] + df_update.loc[poses_to_display,['X','Y','Z','Rx','Ry']], '\n\n')
    print('Change in calculated endpoint positions (5DOF frame): \n',compute_endpoint_errors(df,df_update),'\n\n')
    print('Change in calculated endpoint positions (sMPA frame): \n',compute_endpoint_errors_sMPA_frame(df,df_update),'\n\n')
    print('Verify that all poses intersect at the pupil in the same way before and after the update:')
    print(check_pupil_crossing([df,df_update]),'\n\n')
    print('Distance from pose focus to sMPA: \n\n',compute_distance_to_sMPA(df),'\n\n')  
    

# 2 Write the new poses to an Excel file in the current directory

In [19]:
write_to_excel = True

if write_to_excel == True:
    write_new_poses_to_Excel('test output (from ADM update).xlsx','temporary plateau name',columns=columns,GSA_angle_WCS_deg=GSA_angle_WCS_deg,
                             df=df,df_encoders=pose_encoders,df_update=df_update,df_update_encoders=df_update_encoder_space,
                            focal_length=focal_length,p_null_PAT_baseline_encoder_original=p_null_PAT_baseline_encoder_original,
                            p_null_PAT_update_encoder_original=p_null_PAT_update_encoder_original)

**Writing to Excel complete**
